# Диагностика модели: тесты на гетероскедастичность

In [3]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.stats.api import het_breuschpagan, het_white # тесты на гетероскедастичность

from scipy.stats import chi2 # chi2-распределение и критические значения

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Диагностика модели: тесты на гетероскедастичность</span></i></b><br>

Рассмотрим линейную регрессию 

$$
\begin{aligned}
	y_i&=x_i^\top\beta+u_i & i&=1,\ldots,n
\end{aligned}
$$
с условиями на ошибку
* $\mathsf{E}(u_i|X)=0$ (экзогенность регрессоров)
* $cov(u_i,u_j|X)=0$ при $i\ne j$ (отсутствие серийной корреляции)

Будем тестировать условие гетероскедастичности ошибки, т.е.

$$
	H_0:Var(u_i|X)=\sigma^2
$$

при различных альтернативах

**<span style="color:purple">BP-тест:</span>** рассмотрим модель гетероскедастичности в предположении, что дисперсия ошибки может
зависеть от переменных $z=\begin{pmatrix} 1 & z_1 & \cdots & z_K \end{pmatrix}$ для некоторой гладкой функции $f(\cdot)$

$$
	Var(u|X)=f(z^\top \gamma)=f(\gamma_0+\gamma_1z_1+\cdots+\gamma_K z_K)
$$

В качестве переменных $z$ обычно выступают регрессоры модели.

Для этой модели тестируется 

$$
	H_0':\gamma_1=\cdots=\gamma_K=0
$$

Тест основан на вспомогательной (неинтерпретируемой!) регрессии для OLS-остатков

$$
\begin{aligned}
	e^2\;&\text{на}\; z & &R^2_{aux}
\end{aligned}
$$

__Два подхода к тестированию__
* тестовая статистика $LM=nR^2_{aux}$, критическое значение $\chi^2_{cr}=\chi^2_{df=K}(\alpha)$ ($K$ - число предикторов во вспомогательной регрессии)
* F-тест на значимость вспомогательной регрессии

__Вывод:__ Тест указывает на гетероскедастичность (отвергаем $H_0,H'_0$) при $LM>\chi^2_{cr}, F>F_{cr},P<\alpha$. 

__Замечание:__ 

* гетероскедастичность м.б. следствием неоднородности данных. В этом случаем в качестве $z$ можно выбирать "переменные масштаба". Такие переменные могут быть среди регрессоров модели.
* используем метод [`statsmodels.stats.diagnostic.het_breuschpagan(resid, exog_het)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.het_breuschpagan.html#statsmodels.stats.diagnostic.het_breuschpagan)
* где `resid` -  остатки модели
* `exog_het` – матрица наблюдений переменных $z$ (например, матрица регрессионного дизайна $X$ для регрессоров модели)
* методы возвращает четыре числа: $LM$-статистику и её P-значение, $F$-статистику и её P-значение
 

**<span style="color:purple">White-тест:</span>** тест на гетероскедастичность общего вида 

Тест основан на вспомогательной (неинтерпретируемой!) регрессии для OLS-остатков

$$
\begin{aligned}
	e^2\;&\text{на}\; x_j,x_j^2,x_jx_l & &R^2_{aux}
\end{aligned}
$$

__Два подхода к тестированию__
* тестовая статистика $LM=nR^2_{aux}$, критическое значение $\chi^2_{cr}=\chi^2_{df=K}(\alpha)$ ($K$ - число предикторов во вспомогательной регрессии)
* F-тест на значимость вспомогательной регрессии

__Вывод:__ Тест указывает на гетероскедастичность (отвергаем $H_0$) при $LM>\chi^2_{cr}, F>F_{cr},P<\alpha$. 

__Замечание:__ 

* используем метод [`statsmodels.stats.diagnostic.het_white(resid, exog)`](https://www.statsmodels.org/stable/generated/statsmodels.stats.diagnostic.het_white.html#statsmodels.stats.diagnostic.het_white)
* где `resid` -  остатки модели
* `exog` – матрица наблюдений переменных $z$ (например, матрица регрессионного дизайна $X$ для регрессоров модели)
* методы возвращает четыре числа: $LM$-статистику и её P-значение, $F$-статистику и её P-значение

</div>

Для датасета `Labour` рассмотрим регрессию 

$$
	\log(output)\;\text{на}\log(capital),\log(labour),\log^2(capital),\log^2(capital)
$$

In [4]:
# загрузим данные
df = pd.read_csv('Labour.csv')

In [5]:
# спецификация модели
mod = ols(formula='np.log(output)~1+np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res = mod.fit()

## BP-тест

Тестируем зависимость дисперсии ошибки от регрессоров модели. В качестве аргумента `exog_het` возьмём матрицу регрессивного дизайна $X$

In [6]:
lm, lm_pvalue, fvalue, f_pvalue = het_breuschpagan(resid=res.resid, exog_het=mod.exog)
# LM-статистика и её P-значение 
lm, lm_pvalue

(44.53371850842652, 4.970083203925146e-09)

## Критическое значение $\chi^2$-распределения

In [7]:
# Задаём уровень значимости
sign_level = 0.05
# Критическое значение распределения chi2
chi2.ppf(q=1-sign_level, df=mod.df_model)

9.487729036781154

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<span style="color: purple"><i><b>Тест указывает на гетероскедастичность </i></b></span> (отвергаем $H_0,H'_0$) т.к. $LM>\chi^2_{cr}$ или $P<\alpha$.

</div>

## White-тест

Тестируем зависимость дисперсии ошибки от регрессоров модели

In [8]:
lm, lm_pvalue, fvalue, f_pvalue = het_white(resid=res.resid, exog=mod.exog)
# LM-статистика и её P-значение 
lm, lm_pvalue

(75.45405182710691, 3.0150449532098e-11)

## Критическое значение $\chi^2$-распределения

In [9]:
# Задаём уровень значимости
sign_level = 0.05
# Найдем степени свободы
k = mod.df_model # число регрессоров
df_white = 2 * k + k * (k-1) / 2
# Критическое значение распределения chi2
chi2.ppf(q=1-sign_level, df=df_white)

23.684791304840576

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<span style="color: purple"><i><b>Тест указывает на гетероскедастичность </i></b></span> (отвергаем $H_0,H'_0$) т.к. $LM>\chi^2_{cr}$ или $P<\alpha$.

</div>